In [16]:
import pandas as pd
import numpy as np

In [18]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/users_final_numeric.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/products_final_numeric.csv')

In [19]:
users_final.shape

(80000, 17)

In [20]:
products_final.shape

(19696, 33)

## Building baseline 2 tower model

### The first issue is that for each training batch, we need to have the same amount of user-item pairs as input. This means we need to use some sort of sampling for each batch in order to make sure they're both the same size.

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot, Attention
from tensorflow.keras.models import Model

In [38]:
user_input = Input(shape=(16,), name='user_input')
item_input = Input(shape=(31,), name='item_input')

In [55]:
#Changed from baseline
user_tower = Dense(128, activation='relu')(user_input)
user_tower = Attention()([user_tower, user_tower])
user_tower = Dense(64, activation='relu')(user_tower)

In [56]:
item_tower = Dense(128, activation='relu')(item_input)
item_tower = Attention()([item_tower, item_tower])
item_tower = Dense(64, activation='relu')(item_tower)

In [57]:
dot_product = Dot(axes=1)([user_tower, item_tower])

In [58]:
model = Model(inputs=[user_input, item_input], outputs=dot_product)

In [59]:
model.compile(optimizer='adam', loss='mse')

In [60]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 128)       │      2,176 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 128)       │      4,096 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_20        │ (None, 128)       │          0 │ dense_23[0][0],   │
│ (Attention)         │                   │            │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_21        │ (None, 128)       │          0 │ dense_25[0][0],   │
│ (Attention)         │                   │            │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      8,256 │ attention_20[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 64)        │      8,256 │ attention_21[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_5 (Dot)         │ (None, 1)         │          0 │ dense_24[0][0],   │
│                     │                   │            │ dense_26[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,784 (89.00 KB)

 Trainable params: 22,784 (89.00 KB)

 Non-trainable params: 0 (0.00 B)

### Formatting inputs

In [61]:
    def create_labels_and_train(users_df, products_df, model, batch_size, num_epochs):
        for epoch in range(num_epochs):
            # generate random user-item pairs through random indices for each batch
            user_indices = np.random.randint(0, len(users_df), size=batch_size)
            product_indices = np.random.randint(0, len(products_df), size=batch_size)

            # extract the data
            user_data = users_df.iloc[user_indices]#.copy()  # copy to avoid SettingWithCopyWarning
            product_data = products_df.iloc[product_indices]#.copy()

            # we will be creating target similarity labels
            target_similarity = []

            # loop through user and product indices to create labels
            for user_idx, product_idx in zip(user_indices, product_indices):
                user_product_id = users_df.iloc[user_idx]['product_id']
                item_product_id = products_df.iloc[product_idx]['product_id']

                # if the user and item product id match, it means the user purchased the product
                # otherwise, there is no interaction and the target similarity would be 0
                target_similarity.append(1 if user_product_id == item_product_id else 0)

            # convert to a numpy array
            target_similarity = np.array(target_similarity)

            # drop 'product_id' from both dataframes
            user_data = user_data.drop(columns=['product_id'])
            product_data = product_data.drop(columns=['product_id', 'name_embedding'])


            # train the model with the pairs
            model.fit([user_data.values, product_data.values], target_similarity, epochs=1, batch_size=batch_size)

    # parameters
    batch_size = 500
    num_epochs = 25

    create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)


c:\Users\asmi_\Asmi's Documents\Academics\2022-26_UCLA\Projects\amex\.venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


ValueError: Exception encountered when calling Attention.call().

[1mDimension must be 2 but is 3 for '{{node functional_5_1/attention_21_1/transpose}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32](functional_5_1/dense_25_1/Relu, functional_5_1/attention_21_1/transpose/perm)' with input shapes: [500,128], [3].[0m

Arguments received by Attention.call():
  • inputs=['tf.Tensor(shape=(500, 128), dtype=float32)', 'tf.Tensor(shape=(500, 128), dtype=float32)']
  • mask=['None', 'None']
  • training=True
  • return_attention_scores=False
  • use_causal_mask=False